In [3]:
from multiprocessing import Pool
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd
import re


data = pd.read_csv('CorEA-message-level_clean2.csv', sep='\t',quotechar="'",
                  engine='python')
data=data.head(12)


# In[6]:

lex = pd.read_csv('ITALconvertcsv.csv', delimiter=",", quotechar='"', header=0)
lex.head(5)
#print(lex[0:5])
#print(list(lex))


# In[20]:

# In[7]:

new_lex=pd.concat([lex['Lemma/_writtenForm'],lex['Sense/Confidence/_score'], lex['Sense/Sentiment/_polarity'], ],axis=1)

halfindex=int((len(new_lex.index))/2)
halfword=(new_lex['Lemma/_writtenForm'][halfindex])
halfplusword=(new_lex['Lemma/_writtenForm'][halfindex+1])
while (halfword[0]==halfplusword[0]):
    halfindex+=1
    halfplusindex=halfindex+1
    halfword=(new_lex['Lemma/_writtenForm'][halfindex])
    halfplusword=(new_lex['Lemma/_writtenForm'][halfplusindex])

#print(halfword)
#print(halfindex)

#print(halfplusword)
#print(halfplusindex)

uphalfLex=new_lex.ix[:halfindex]
lowhalfLex=new_lex.ix[halfplusindex:]
uphalfLex.reset_index(drop=True, inplace=True)
lowhalfLex.reset_index(drop=True, inplace=True)
#print(uphalfLex)
#print(lowhalfLex)


# In[56]:

upquarterindex=int((len(uphalfLex.index))/2)
upquarterword=(uphalfLex['Lemma/_writtenForm'][upquarterindex])
upquarterplusword=(uphalfLex['Lemma/_writtenForm'][upquarterindex+1])
while (upquarterword[0]==upquarterplusword[0]):
    upquarterindex+=1
    upquarterplusindex=upquarterindex+1
    upquarterword=(uphalfLex['Lemma/_writtenForm'][upquarterindex])
    upquarterplusword=(uphalfLex['Lemma/_writtenForm'][upquarterplusindex])

#print(upquarterword)
#print(upquarterplusword)
upupquarterLex=uphalfLex.ix[:upquarterindex]
uplowquarterLex=uphalfLex.ix[upquarterplusindex:]

upupquarterLex.reset_index(drop=True, inplace=True)
uplowquarterLex.reset_index(drop=True, inplace=True)
#print(upupquarterLex)
#print(uplowquarterLex)


# In[59]:

lowquarterindex=int((len(lowhalfLex.index))/2)

lowquarterword=(lowhalfLex['Lemma/_writtenForm'][lowquarterindex])
lowquarterplusword=(lowhalfLex['Lemma/_writtenForm'][lowquarterindex+1])
while (lowquarterword[0]==lowquarterplusword[0]):
    lowquarterindex+=1
    lowquarterplusindex=lowquarterindex+1
    lowquarterword=(lowhalfLex['Lemma/_writtenForm'][lowquarterindex])
    lowquarterplusword=(lowhalfLex['Lemma/_writtenForm'][lowquarterplusindex])

#print(lowquarterword)
#print(lowquarterplusword)
lowupquarterLex=lowhalfLex.ix[:lowquarterindex]
lowlowquarterLex=lowhalfLex.ix[lowquarterplusindex:]

lowupquarterLex.reset_index(drop=True, inplace=True)
lowlowquarterLex.reset_index(drop=True, inplace=True)
#print(lowupquarterLex)
#print(lowlowquarterLex)


# In[ ]:

# In[11]:
cellvavlist=[]
leftrightlist=[]
midlist=[]
#for cell in (data['text']):
 #   for word in (cell.split()):


rightlist=[]
leftlist=[]



#for each cell in the text column of data

def lexiconclassifier(line):
    cell=(line['6text'])
    mid=(line['#0Mid'])
    new_lex=pd.DataFrame()
    cellrating=0
    wordcount=0
    lr_classifier=0
    LeftOrRight="None"

    #for reach word in cell
    for word in (cell.split()):
        word=re.sub("[^A-Za-z|-]","", word)
        #print(word)
        
        
        ratingfactor=0
        #iterates through the lexicon
        if word in rightlist and lr_classifier==0:
            #print(word)
            lr_classifier="right"
            #print(lr_classifier)
        if word in rightlist and lr_classifier=="right":
            #print(word)
            lr_classifier="right"
            #print(lr_classifier)
        if word in rightlist and lr_classifier=="left":
            #print(word)
            lr_classifier="None"
            #print(lr_classifier)
        if word in rightlist and lr_classifier=="left":
            #print(word)
            lr_classifier="left"
            #print(lr_classifier)
        if word in leftlist and lr_classifier==0:
            #print(word)
            lr_classifier="left"
            #print(lr_classifier)
        if word in leftlist and lr_classifier=="right":
            #print(word)
            lr_classifier="None"
        try:
            if word[0]<=halfplusword[0]:
                if word[0]<=upquarterplusword[0]:
                    new_lex=upupquarterLex
                else:
                    new_lex=uplowquarterLex
            else:
                if word[0]<=lowquarterplusword[0]:
                    new_lex=lowupquarterLex
                else:
                    new_lex=lowlowquarterLex
        except LookupError:
            pass

        for index, row in (new_lex).iterrows():
            #if word is found in the lexicon
            #print str.startswith( 'this' )
            if word.startswith(row["Lemma/_writtenForm"]):
                wordcount+=1
                #classifier
                if row["Sense/Sentiment/_polarity"]=="neutral":
                    ratingfactor=0
                if row["Sense/Sentiment/_polarity"]=="positive":
                    ratingfactor=1
                if row["Sense/Sentiment/_polarity"]=="negative":
                    ratingfactor=-1
                rating=(row["Sense/Confidence/_score"])
                cellrating=cellrating+(ratingfactor*rating)
    if wordcount==0:
        cellaverage=0
    else:
        cellaverage=(cellrating/wordcount)
    print(cellaverage)
    print(mid)
    element={}
    element['cellaverage']=cellaverage
    element['mid']=mid
   
    
    return(element)
   
num_cores = multiprocessing.cpu_count()

print(num_cores)

result=Parallel(n_jobs=num_cores)(delayed(lexiconclassifier)(line) for index,line in (data[['#0Mid','6text']]).iterrows())


print(result)
#extrarow = pd.DataFrame(cellvavlist)
#extrarow2=pd.DataFrame(leftrightlist)

#output=pd.concat([data, extrarow, extrarow2],axis=1)
#output.to_csv('Corea.csv', encoding='utf-8')


# In[ ]:




# In[ ]:


4
0.0
18031
0.20833333333325
18106
0.14285714285728573
18087
0
18145
0.05
18006
0.36111111111116667
18124
0.22916666666649999
18160
0.2666666666668
18189
-0.08974358974353845
18086
0.05
18179
-0.09375
18147
0.05555555555555555
18191
[{'cellaverage': 0.05, 'mid': 18006}, {'cellaverage': 0.0, 'mid': 18031}, {'cellaverage': -0.08974358974353845, 'mid': 18086}, {'cellaverage': 0.14285714285728573, 'mid': 18087}, {'cellaverage': 0.20833333333325, 'mid': 18106}, {'cellaverage': 0.36111111111116667, 'mid': 18124}, {'cellaverage': 0, 'mid': 18145}, {'cellaverage': -0.09375, 'mid': 18147}, {'cellaverage': 0.22916666666649999, 'mid': 18160}, {'cellaverage': 0.05, 'mid': 18179}, {'cellaverage': 0.2666666666668, 'mid': 18189}, {'cellaverage': 0.05555555555555555, 'mid': 18191}]


In [16]:
test=pd.DataFrame.from_dict(result, orient='columns', dtype=None)

In [17]:
test.head(10)

,cellaverage,mid
0,0.050000,18006
1,0.000000,18031
2,-0.089744,18086
3,0.142857,18087
4,0.208333,18106
5,0.361111,18124
6,0.000000,18145
7,-0.093750,18147
8,0.229167,18160
9,0.050000,18179


In [20]:
test.to_csv("ItaloLexRate.csv", index=False)